In [ ]:
import os
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import brainscore.benchmarks as bench
from brainscore.metrics.regression import linear_regression, ridge_regression
#from bonnerlab_brainscore.benchmarks.object2vec import Object2VecEncoderBenchmark
from model_tools.activations.pca import LayerPCA
from model_tools.brain_transformation.neural import LayerScores
from activation_models.generators import get_activation_models
from custom_model_tools.hooks import GlobalMaxPool2d, GlobalAvgPool2d, RandomSpatial
from utils import timed, id_to_properties
import logging

logging.basicConfig(level=logging.INFO)


scores = pd.DataFrame()
for model, layers in get_activation_models():
    layers_scores = ridge_encoder()
    score = scores.append(layer_scores)
scores.to_csv(f'results/encoding_Eig-OLS|benchmark:{benchmark._identifier}|pooling:{pooling}.csv')

def ridge_encoder():
    


@timed
def main(benchmark, pooling, debug=False):
    save_path = f'results/encoding_Eig-OLS|benchmark:{benchmark._identifier}|pooling:{pooling}.csv'
    if os.path.exists(save_path):
        print(f'Results already exists: {save_path}')
        return
    
    scores = pd.DataFrame()
    for model, layers in get_activation_models():
        layer_scores = fit_encoder(benchmark, model, layers, pooling)
        scores = scores.append(layer_scores)
        if debug:
            break
    if not debug:
        scores.to_csv(save_path, index=False)


def fit_encoder(benchmark, model, layers, pooling, hooks=None):
    """Fit layers one at a time to save on memory"""

    layer_scores = pd.DataFrame()
    model_identifier = model.identifier
    model_properties = id_to_properties(model_identifier)
    
    for layer in layers:
        if pooling == 'max':
            handle = GlobalMaxPool2d.hook(model)
            model.identifier = model_identifier + f'|lyr:{layer}|pooling:max'
        elif pooling == 'avg':
            handle = GlobalAvgPool2d.hook(model)
            model.identifier = model_identifier + f'|layer:{layer}|pooling:avg'
        elif pooling == 'random_spatial':
            handle = RandomSpatial.hook(model)
            model.identifier = model_identifier + f'|layer:{layer}|pooling:random_spatial'
        elif pooling == 'none':
            handle = LayerPCA.hook(model, n_components=1000)
            model.identifier = model_identifier + f'|layer:{layer}|pooling:none|n_components:1000'
        

        handles = []
        if hooks is not None:
            handles = [cls.hook(model) for cls in hooks]
            
        logging.info(model.identifier)
        logging.info(layer)
        model_scores = LayerScores(model_identifier=model.identifier,
                                   activations_model=model,
                                   visual_degrees=8)
        score = model_scores(benchmark=benchmark, layers=[layer], prerun=True)
        handle.remove()

        for h in handles:
            h.remove()

        if 'aggregation' in score.dims:
            score = score.to_dataframe(name='').unstack(level='aggregation').reset_index()
            score.columns = ['layer', 'score', 'score_error']
        else:
            score = score.to_dataframe(name='').reset_index()
            score.columns = ['layer', 'score']

        layer_scores = layer_scores.append(score)

    layer_scores = layer_scores.assign(**model_properties)
    return layer_scores


def get_benchmark(benchmark, region, regression, data_dir):
    if benchmark == 'majajhong2015':
        assert region in ['IT', 'V4']
        identifier = f'dicarlo.MajajHong2015public.{region}-pls'
        benchmark = bench.load(identifier)
        if regression == 'lin':
            benchmark._identifier = benchmark.identifier.replace('pls', 'lin')
            benchmark._similarity_metric.regression = linear_regression()
            #benchmark._similarity_metric.regression._regression.alpha = 0.1
        elif regression == 'l2':
            benchmark._identifier = benchmark.identifier.replace('pls', 'l2')
            benchmark._similarity_metric.regression = ridge_regression()
    elif benchmark == 'freeman2013':
        assert region == 'V1'
        identifier = f'movshon.FreemanZiemba2013public.{region}-pls'
        benchmark = bench.load(identifier)
        if regression == 'lin':
            benchmark._identifier = benchmark.identifier.replace('pls', 'lin')
            benchmark._similarity_metric.regression = linear_regression()
            benchmark._similarity_metric.regression._regression.alpha = 0.1
        elif regression == 'l2':
            benchmark._identifier = benchmark.identifier.replace('pls', 'l2')
            benchmark._similarity_metric.regression = ridge_regression()
    elif benchmark == 'object2vec':
        if region == 'all':
            region = None
        regions = region if region is None or ',' not in region else region.split(',')
        #benchmark = Object2VecEncoderBenchmark(data_dir=data_dir, regions=regions, regression=regression)
    else:
        raise ValueError(f'Unknown benchmark: {benchmark}')
    return benchmark


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Fit encoding models to a neural dataset')
    parser.add_argument('--bench', type=str, default='majajhong2015',
                        choices=['majajhong2015', 'freeman2013', 'object2vec'],
                        help='Neural benchmark dataset to fit')
    parser.add_argument('--region', type=str, default='IT',
                        help='Region(s) to fit. Valid region(s) depend on the neural benchmark')
    parser.add_argument('--regression', type=str, default='pls',
                        choices=['pls', 'lin', 'l2'],
                        help='Partial-least-squares or ordinary-least-squares for fitting')
    parser.add_argument('--data_dir', type=str, default=None,
                        help='Data directory for neural benchmark (only required for "object2vec")')
    parser.add_argument('--pooling', dest='pooling', type=str,
                        choices=['max','avg','none', 'random_spatial'],
                        help='Choose global max-pooling, global avg-pooling, no pooling, or random spatial positions prior to fitting')
    parser.add_argument('--debug', action='store_true',
                        help='Just run a single model to make sure there are no errors')
    args = parser.parse_args()

    benchmark = get_benchmark(benchmark=args.bench, region=args.region,
                              regression=args.regression, data_dir=args.data_dir)
    main(benchmark=benchmark, pooling=args.pooling, debug=args.debug)

#pooling
#fit_encoding_models.py def fit_encoder

In [ ]:
p = '/home/wtownle1/dimensionality_powerlaw/activation_models/AtlasNet/'
import sys
sys.path.append(p)
from activation_models.AtlasNet.model_2L import EngineeredModel2L

model = EngineeredModel2L.Build(filters_2=1000)

images = 1 #load dicarlo.hvm-public

#X = dataset.iloc[:, 2:-1].values
X = model(images)
#y = dataset. iloc [:, 1].values
y = 2 #load neural responses (assy_dicarlo_MajajHOng2015)



In [6]:
import pandas as pd
majaj_csv = pd.read_csv('/data/shared/brainio/brain-score/image_dicarlo_hvm-public.csv')
majaj_csv[majaj_csv['image_file_name'] == "Pear_obj_rx-92.621_ry-00.595_rz-17.488_tx-00.213_ty-00.599_s+00.907_9edd70d8418078fbb25cca46e88d5436983a265f_256x256.png"]
#image_file_name

,id,background_id,s,image_id,image_file_name,filename,rxy,tz,category_name,rxz_semantic,ty,ryz,object_name,variation,size,rxy_semantic,ryz_semantic,rxz
641,642,9edd70d8418078fbb25cca46e88d5436983a265f,1.295714,c41c369d7cde564cdb841d66c69cd8ffe025dc12,Pear_obj_rx-92.621_ry-00.595_rz-17.488_tx-00.2...,Pear_obj_rx-92.621_ry-00.595_rz-17.488_tx-00.2...,0.027237,-0.599,Fruits,-2.620859,-0.213,-16.892377,pear,3,256.0,0.027237,-16.892377,-2.620859


In [ ]:
MACAQUES = {0: "Chabo", 1: "Tito"}


def open_majajhong_assembly(macaque: int, roi: str) -> xr.DataArray:
    assembly = DataAssembly(xr.open_dataarray(MH_PATH))
    assembly = average_repetition(assembly)
    assembly = (
        assembly.rename("data")
        .assign_attrs(
            {
                "time_bin": (
                    assembly["time_bin_start"].values[0],
                    assembly["time_bin_end"].values[0],
                ),
                "animal": MACAQUES[macaque],
            }
        )
        .isel(time_bin=0, drop=True)
        .isel({"neuroid": assembly["animal"].values == MACAQUES[macaque]})
    )
    assembly = assembly.isel({"neuroid": assembly["region"].values == roi}).transpose(
        "presentation", "neuroid"
    )
    return assembly

In [10]:
#def update_by_email(email=None, **kwargs):
#    print(kwargs)


p = '/home/wtownle1/encoder_dimensionality/'
import sys
sys.path.append(p)

import os
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import brainscore.benchmarks as bench
from brainscore.metrics.regression import linear_regression, ridge_regression
#from bonnerlab_brainscore.benchmarks.object2vec import Object2VecEncoderBenchmark
from model_tools.activations.pca import LayerPCA
from model_tools.brain_transformation.neural import LayerScores
from activation_models.generators import get_activation_models
from custom_model_tools.hooks import GlobalMaxPool2d #, GlobalAvgPool2d, RandomSpatial
from utils import timed, id_to_properties
import logging

#logging.basicConfig(level=logging.INFO)



def get_benchmark(benchmark, region, regression, data_dir):
    if benchmark == 'majajhong2015':
        assert region in ['IT', 'V4']
        identifier = f'dicarlo.MajajHong2015public.{region}-pls'
        benchmark = bench.load(identifier)
        if regression == 'lin':
            benchmark._identifier = benchmark.identifier.replace('pls', 'lin')
            benchmark._similarity_metric.regression = linear_regression()
            #benchmark._similarity_metric.regression._regression.alpha = 0.1
        elif regression == 'l2':
            alpha = 1
            benchmark._identifier = benchmark.identifier.replace('pls', f'ridge_alpha={alpha}')
            benchmark._similarity_metric.regression = ridge_regression(sklearn_kwargs = {'alpha':alpha})
            benchmark._similarity_metric.regression = ridge_regression()
    elif benchmark == 'freeman2013':
        assert region == 'V1'
        identifier = f'movshon.FreemanZiemba2013public.{region}-pls'
        benchmark = bench.load(identifier)
        if regression == 'lin':
            benchmark._identifier = benchmark.identifier.replace('pls', 'lin')
            benchmark._similarity_metric.regression = linear_regression()
            benchmark._similarity_metric.regression._regression.alpha = 0.1
        elif regression == 'l2':
            benchmark._identifier = benchmark.identifier.replace('pls', 'l2')
            benchmark._similarity_metric.regression = ridge_regression()
    elif benchmark == 'object2vec':
        if region == 'all':
            region = None
        regions = region if region is None or ',' not in region else region.split(',')
        #benchmark = Object2VecEncoderBenchmark(data_dir=data_dir, regions=regions, regression=regression)
    else:
        raise ValueError(f'Unknown benchmark: {benchmark}')
    return benchmark


#if __name__ == '__main__':
#    parser = argparse.ArgumentParser(description='Fit encoding models to a neural dataset')
#    parser.add_argument('--bench', type=str, default='majajhong2015',
#                        choices=['majajhong2015', 'freeman2013', 'object2vec'],
#                        help='Neural benchmark dataset to fit')
#    parser.add_argument('--region', type=str, default='IT',
#                        help='Region(s) to fit. Valid region(s) depend on the neural benchmark')
#    parser.add_argument('--regression', type=str, default='pls',
#                        choices=['pls', 'lin', 'l2'],
#                        help='Partial-least-squares or ordinary-least-squares for fitting')
#    parser.add_argument('--data_dir', type=str, default=None,
#                        help='Data directory for neural benchmark (only required for "object2vec")')
#    parser.add_argument('--pooling', dest='pooling', type=str,
#                        choices=['max','avg','none', 'random_spatial'],
#                        help='Choose global max-pooling, global avg-pooling, no pooling, or random spatial positions prior to fitting')
#    parser.add_argument('--debug', action='store_true',
#                        help='Just run a single model to make sure there are no errors')
#    args = parser.parse_args()
#

benchmark = get_benchmark(benchmark='majajhong2015', region='IT', regression='l2', data_dir=None)
#main(benchmark=benchmark, pooling=args.pooling, debug=args.debug)

print(benchmark.identifier)


Loading catalog from entrypoints
Loading lookup from /home/wtownle1/.conda/envs/encoder-dimensionality/lib/python3.7/site-packages/brainscore/lookup.csv


In [15]:
print(benchmark._similarity_metric.regression._regression.alpha)
benchmark._identifier = benchmark.identifier.replace('pls', f'ridge_alpha={alpha}')
                benchmark._similarity_metric.regression = ridge_regression(sklearn_kwargs = {'alpha':alpha})

1.0


In [ ]:
def pls_regression(regression_kwargs=None, xarray_kwargs=None):
    regression_defaults = dict(n_components=25, scale=False)
    regression_kwargs = {**regression_defaults, **(regression_kwargs or {})}
    regression = PLSRegression(**regression_kwargs)
    xarray_kwargs = xarray_kwargs or {}
    regression = XarrayRegression(regression, **xarray_kwargs)
    return regression

def ridge_regression(regression_kwargs=None, xarray_kwargs=None):
    regression_kwargs = regression_kwargs or {}
    regression = Ridge(**regression_kwargs)
    xarray_kwargs = xarray_kwargs or {}
    regression = XarrayRegression(regression, **xarray_kwargs)
    return regression